In [28]:
# 导入环境变量
from dotenv import load_dotenv
load_dotenv()

True

In [37]:
import os
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

print("Loading.....")
loader = PyPDFLoader("arkUI自定义组件生命周期.pdf")
docs = loader.load()
print("Loadded....")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),persist_directory="./vector_store")


Loading.....
Loadded....


In [38]:
print(docs[0].page_content)

您当前正在 浏 览 HarmonyOS 最 新 文 档 ， 覆 盖 已 发 布 的 所 有API 版本 ， 可在 API 参考 中筛 选 您 使 用 的API 版本。 详 细 的 版本 配 套 关系请 参考版本说明。筛 选目 录下 的 文 档 标 题
入门
开发
工 具
测试
基 础 入门
应 用开发 准 备
应 用 框 架
Ability  Kit （ 程 序 框 架 服务）
Accessibility  Kit  ( 无 障 碍 服务 )
ArkData （方 舟 数 据 管 理）
ArkTS （方 舟 编程 语 言 ）
ArkUI （方 舟 UI 框 架 ）
ArkUI 简介
UI 开发  (ArkTS 声 明式开发 范 式 )
UI 开发（ ArkTS 声 明式开发 范 式） 概 述
学 习 UI 范 式 基 本 语 法
基 本 语 法 概 述
声 明式 UI 描述
自定 义 组 件
创建 自定 义 组 件
自定 义 组 件 生 命 周 期
自定 义 组 件 的 自定 义 布 局
自定 义 组 件 成 员 属 性 访 问 限 定 符 使
用 限 制
组 件 扩 展
@ Styles 装 饰 器 ： 定 义 组 件重 用 样 式
@ Extend 装 饰 器 ： 定 义 扩 展 组 件样 式
stateStyles： 多 态样 式
@ AnimatableExtend 装 饰 器 ： 定 义
可动 画属 性
@ Require 装 饰 器 ： 校 验构造 传 参
@ Reusable 装 饰 器 ： 组 件复 用
学 习 UI 范 式 状 态管 理
学 习 UI 范 式 渲 染 控 制
设置 组 件导 航 和页面 路由
组 件布 局
列 表与 网 格
使 用文本
媒 体 展示
表 单选 择
添 加 组 件
使 用 弹窗
几 何 图 形 绘 制
添 加 交 互 响 应
使 用动 画
使 用自定 义 能力
UI 国 际 化
无 障 碍 与 适 老 化
主题 设置
UI 系 统场 景 化能力
UI 开发  ( 基 于 N DK 构 建 UI)
UI 开发  ( 兼 容 JS的 类 Web 开发 范 式 )
UI 开发 调试调优
窗 口管 理
屏 幕 管 理
ArkWeb （方 舟 Web ）
Background  Tasks  Kit （后 台任 务开发服
务）


In [39]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
print("RAG....")
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

RAG....


In [40]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
 
 
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [41]:
rag_chain.invoke("告诉我arkUI自定义组件的生命周期")

'ArkUI自定义组件的生命周期包括以下几个关键阶段：首先是`aboutToAppear`，在组件创建后但在执行`build`函数之前调用；接着是`onDidBuild`，在组件首次渲染完成后调用；最后是`aboutToDisappear`，在组件销毁之前调用。开发者应注意在这些生命周期函数中不修改状态变量，以避免不稳定的UI表现。'